Dayton Berezoski 228005087

Note: I am not allowed to distribute the data from the MIMIC-IV and eICU databases, so they are not included in the submission. However, the respective SQL queries to get the exact same csv files will be included with this submission if you would like to run the code yourself and have access to these databases. As such the outputs for processing the data will be removed as well.

Imports

In [91]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
import sklearn.metrics as metrics
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data_utils


In [92]:
onehot_encoder = OneHotEncoder(sparse=False)
label_encoder = LabelEncoder()

In [93]:
def scores (pred, actual):
    fpr, tpr, threshold = metrics.roc_curve(actual, pred, pos_label = 1)
    aucroc = metrics.auc(fpr,tpr)
    exp_deaths = np.count_nonzero(pred == 1)
    act_deaths = actual.value_counts()[1]
    smr = act_deaths/exp_deaths

    print('AUC-ROC:', aucroc)
    print('SMR:', smr)

MIMIC-IV (train) data loading and cleaning

In [ ]:
data_m = pd.read_csv('MIMIC_IV.csv')
data_m

In [ ]:
data_m['death'] = data_m['death'].notnull().astype('int')

data_m

In [ ]:
Y_m = data_m['death']
Y_m

In [97]:
x_m = data_m.drop(['subject_id', 'hadm_id', 'death'], axis=1)
categorical_m = x_m[['gender', 'eye_resp_f', 'verbal_resp_f', 'motor_resp_f', 'eye_resp_l', 'verbal_resp_l', 'motor_resp_l']]
numeric_m = x_m.drop(['gender', 'eye_resp_f', 'verbal_resp_f', 'motor_resp_f', 'eye_resp_l', 'verbal_resp_l', 'motor_resp_l'], axis=1)

In [ ]:
numeric_m

In [ ]:
categorical_m = pd.get_dummies(categorical_m, columns=['gender'], drop_first=True)
categorical_m = pd.get_dummies(categorical_m, columns=['eye_resp_f'], drop_first=True)
categorical_m = pd.get_dummies(categorical_m, columns=['verbal_resp_f'], drop_first=True)
categorical_m = pd.get_dummies(categorical_m, columns=['motor_resp_f'], drop_first=True)
categorical_m = pd.get_dummies(categorical_m, columns=['eye_resp_l'], drop_first=True)
categorical_m = pd.get_dummies(categorical_m, columns=['verbal_resp_l'], drop_first=True)
categorical_m = pd.get_dummies(categorical_m, columns=['motor_resp_l'], drop_first=True)

categorical_m

eICU (test) data loading and cleaning

In [ ]:
data_e = pd.read_csv('eICU.csv')
data_e

In [ ]:
gen_norm = {'Male' : 'M', 'Female' : 'F', 'Unknown' : float('NaN')}
death_norm = {'Death' : 1, 'Home' : 0, 'Rehabilitation' : 0, 'Skilled Nursing Facility' : 0, 'Other' : 0, 'Other External' : 0, 'Other Hospital' : 0, 'Nursing Home' : 0, '' : 0}

data_e['gender'].replace(gen_norm, inplace=True)
data_e['death'].replace(death_norm, inplace=True)
data_e['tem_f'] = (data_e['tem_f'] * (9/5)) + 32
data_e['tem_l'] = (data_e['tem_l'] * (9/5)) + 32
data_e['age'] = pd.to_numeric(data_e['age'], errors='coerce')
data_e = data_e.dropna().reset_index()
data_e = data_e.drop(['index'], axis=1)

data_e

In [ ]:
Y_e = data_e['death'].astype(int)
Y_e

In [103]:
x_e = data_e.drop(['patientunitstayid', 'death'], axis=1)
categorical_e = x_e[['gender', 'eye_resp_f', 'verbal_resp_f', 'motor_resp_f', 'eye_resp_l', 'verbal_resp_l', 'motor_resp_l']]
numeric_e = x_e.drop(['gender', 'eye_resp_f', 'verbal_resp_f', 'motor_resp_f', 'eye_resp_l', 'verbal_resp_l', 'motor_resp_l'], axis=1)

In [ ]:
numeric_e

In [ ]:
categorical_e = pd.get_dummies(categorical_e, columns=['gender'], drop_first=True)
categorical_e = pd.get_dummies(categorical_e, columns=['eye_resp_f'], drop_first=True)
categorical_e = pd.get_dummies(categorical_e, columns=['verbal_resp_f'], drop_first=True)
categorical_e = pd.get_dummies(categorical_e, columns=['motor_resp_f'], drop_first=True)
categorical_e = pd.get_dummies(categorical_e, columns=['eye_resp_l'], drop_first=True)
categorical_e = pd.get_dummies(categorical_e, columns=['verbal_resp_l'], drop_first=True)
categorical_e = pd.get_dummies(categorical_e, columns=['motor_resp_l'], drop_first=True)

categorical_e

Combine numeric values to get the normalized values

In [ ]:
split = len(numeric_m.index)
split

In [ ]:
numeric_tot = numeric_m.append(numeric_e, ignore_index=True, sort=False)
numeric_tot

In [ ]:
numeric_tot = (numeric_tot-numeric_tot.mean())/numeric_tot.std()
numeric_tot

Finalizing dataframes

In [ ]:
numeric_m = numeric_tot.iloc[:split,:]
numeric_m

In [ ]:
X_m = pd.concat([categorical_m, numeric_m], axis = 1)
X_m

In [ ]:
numeric_e = numeric_tot.iloc[split:,:].reset_index()
numeric_e = numeric_e.drop(['index'], axis=1)
numeric_e

In [ ]:
X_e = pd.concat([categorical_e, numeric_e], axis = 1)
X_e

In [ ]:
X_e = X_e[X_m.columns]
X_e

Making sure columns are in the correct order. (might need to increase the number of ouputs allowed to see all of the features)

In [114]:
for col in X_m:
    print(col)

gender_M
eye_resp_f_2
eye_resp_f_3
eye_resp_f_4
verbal_resp_f_2
verbal_resp_f_3
verbal_resp_f_4
verbal_resp_f_5
motor_resp_f_2
motor_resp_f_3
motor_resp_f_4
motor_resp_f_5
motor_resp_f_6
eye_resp_l_2
eye_resp_l_3
eye_resp_l_4
verbal_resp_l_2
verbal_resp_l_3
verbal_resp_l_4
verbal_resp_l_5
motor_resp_l_2
motor_resp_l_3
motor_resp_l_4
motor_resp_l_5
motor_resp_l_6
age
bpm_f
bpm_l
resp_rate_f
tem_f
tem_l
resp_rate_l
O2_sat_f
O2_sat_l
bps_f
bps_l
bpd_f
bpd_l
bpsm_f
bpsm_l


In [115]:
for col in X_e:
    print(col)

gender_M
eye_resp_f_2
eye_resp_f_3
eye_resp_f_4
verbal_resp_f_2
verbal_resp_f_3
verbal_resp_f_4
verbal_resp_f_5
motor_resp_f_2
motor_resp_f_3
motor_resp_f_4
motor_resp_f_5
motor_resp_f_6
eye_resp_l_2
eye_resp_l_3
eye_resp_l_4
verbal_resp_l_2
verbal_resp_l_3
verbal_resp_l_4
verbal_resp_l_5
motor_resp_l_2
motor_resp_l_3
motor_resp_l_4
motor_resp_l_5
motor_resp_l_6
age
bpm_f
bpm_l
resp_rate_f
tem_f
tem_l
resp_rate_l
O2_sat_f
O2_sat_l
bps_f
bps_l
bpd_f
bpd_l
bpsm_f
bpsm_l


Logistic Regression

In [116]:
skf = StratifiedKFold(n_splits = 4)
AUC_ROC_scores = []
SMR_scores = []
i = 1

for train_idx, test_idx in skf.split(X_m,Y_m):
    X_train, X_test = X_m.iloc[train_idx,:], X_m.iloc[test_idx,:]
    y_train, y_test = Y_m.iloc[train_idx], Y_m.iloc[test_idx]
    
    logistic_model = LogisticRegression(max_iter = 100000).fit(X_train, y_train, sample_weight=None)

    pred = logistic_model.predict(X_test)
    fpr, tpr, threshold = metrics.roc_curve(y_test, pred)
    auroc = metrics.auc(fpr,tpr)

    exp_deaths = np.count_nonzero(pred == 1)
    act_deaths = y_test.value_counts()[1]
    smr = act_deaths/exp_deaths

    AUC_ROC_scores.append(auroc)
    SMR_scores.append(smr)
    
    print('Done with', i,'splits')
    i += 1

print()
print('AUC-ROC scores:', AUC_ROC_scores)
print('Mean AUC-ROC score:', np.mean(AUC_ROC_scores))
print('SMR scores:', SMR_scores)
print('Mean SMR score:', np.mean(SMR_scores))

Done with 1 splits
Done with 2 splits
Done with 3 splits
Done with 4 splits

AUC-ROC scores: [0.8743968679490367, 0.8721455696836705, 0.867393385564546, 0.8558037493055971]
Mean AUC-ROC score: 0.8674348931257125
SMR scores: [1.189463955637708, 1.2199052132701422, 1.231578947368421, 1.2377285851780557]
Mean SMR score: 1.2196691753635818


In [117]:
logistic_model = LogisticRegression(max_iter = 100000).fit(X_m, Y_m, sample_weight=None)

In [118]:
pred_log = logistic_model.predict(X_e)


In [119]:
scores(pred_log, Y_e)

AUC-ROC: 0.6907740874466239
SMR: 1.046468401486989


Gradient Boosted Tree

In [146]:

for i in range(3, 6):
    X_train, X_valid, y_train, y_valid = train_test_split(X_m, Y_m, test_size=0.2, random_state=315)
    GB = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.01, max_depth=i, subsample=0.5, n_iter_no_change=10)
    GB.fit(X_train, y_train)
    y_train_pred = GB.predict_proba(X_train)[:,1]
    y_valid_pred = GB.predict_proba(X_valid)[:,1]
    print('Max Depth:', i)
    print('Train AUC-ROC score:',metrics.roc_auc_score(y_train, y_train_pred))
    print('Validation AUC-ROC score:',metrics.roc_auc_score(y_valid, y_valid_pred))
    print()


Max Depth: 3
Train AUC-ROC score: 0.9732698862543125
Validation AUC-ROC score: 0.9653314174397318

Max Depth: 4
Train AUC-ROC score: 0.9750060008717129
Validation AUC-ROC score: 0.9661369132160071

Max Depth: 5
Train AUC-ROC score: 0.9784176311956357
Validation AUC-ROC score: 0.9663605450096404



In [121]:
GB = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.01, max_depth=i, subsample=0.5, n_iter_no_change=10, verbose=1)
GB.fit(X_m, Y_m)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.6561           0.0136            1.69m
         2           0.6479           0.0122            1.62m
         3           0.6239           0.0119            1.72m
         4           0.6212           0.0109            1.65m
         5           0.6136           0.0102            1.72m
         6           0.5973           0.0097            1.80m
         7           0.6021           0.0090            1.82m
         8           0.5763           0.0086            1.85m
         9           0.5676           0.0084            1.81m
        10           0.5670           0.0078            1.80m
        20           0.4978           0.0054            1.64m
        30           0.4497           0.0040            1.58m
        40           0.4157           0.0032            1.54m
        50           0.3879           0.0025            1.52m
        60           0.3623           0.0022            1.50m
       

GradientBoostingClassifier(learning_rate=0.01, max_depth=5, n_estimators=1000,
                           n_iter_no_change=10, subsample=0.5, verbose=1)

In [122]:
pred_gb = GB.predict(X_e)

In [123]:
scores(pred_gb, Y_e)

AUC-ROC: 0.7607446526990743
SMR: 0.7781617138908086


Neural Net

In [124]:
batch_size = 32

In [125]:
def split_indices(n, val_frac, seed):
    # Determine the size of the validation set
    n_val = int(val_frac * n)
    np.random.seed(seed)
    # Create random permutation between 0 to n-1
    idxs = np.random.permutation(n)
    # Pick first n_val indices for validation set
    return idxs[n_val:], idxs[:n_val]

In [126]:
class NN(nn.Module):
    def __init__(self, input_size, output_size):
        super(NN, self).__init__()
        
        self.fc1 = nn.Linear(input_size,64)
        self.fc2 = nn.Linear(64, output_size)
        
        
    def forward(self, X):
        out = F.relu(self.fc1(X))
        out = F.relu(self.fc2(out))
        return out

In [127]:
network = NN(40,2)

In [128]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [129]:
X_train, X_valid, y_train, y_valid = train_test_split(X_m, Y_m, test_size=0.2, random_state=315)

X_train_tensor = torch.Tensor(X_train.values.astype(np.float32))
y_train_tensor = torch.Tensor(y_train.values.astype(np.float32))
y_train_tensor = y_train_tensor.long()
train_tensor = data_utils.TensorDataset(X_train_tensor, y_train_tensor)
train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)

X_valid_tensor = torch.Tensor(X_valid.values.astype(np.float32))
y_valid_tensor = torch.Tensor(y_valid.values.astype(np.float32))
y_valid_tensor = y_valid_tensor.long()
valid_tensor = data_utils.TensorDataset(X_valid_tensor, y_valid_tensor)
valid_loader = data_utils.DataLoader(dataset = valid_tensor, batch_size = batch_size, shuffle = True)

In [130]:
def train_model(n_epochs, model, train_dl, val_dl, loss_fn, opt_fn, lr):

    train_losses, val_losses, train_accuracies, val_accuracies = [], [], [], []
    
    for epoch in range(n_epochs):
        val_loss = 0
        train_loss = 0.0

        num_corr = 0.0
        num_tot = 0.0

        for i, (x, y) in enumerate(train_dl):
 
            X = x.to(device=device)
            Y = y.to(device=device)
            Y = Y.squeeze()

            opt_fn.zero_grad()
            hyp = model(X)

            cost = loss_fn(hyp, Y)
            cost.backward()

            opt_fn.step()

            pred = hyp.data.max(dim=1)[1]

            num_corr += (pred == y).sum()
            num_tot += pred.size(0)

            train_loss += cost.item()

        train_accuracy = 0

        if num_tot > 0:
            train_accuracy =  num_corr / num_tot  

        num_corr = 0.0
        num_tot = 0.0

        for i, (x, y) in enumerate(val_dl):
            X = x.to(device=device)
            Y = y.to(device=device)
            Y = Y.squeeze()

            hyp = model(X)
            
            cost = loss_fn(hyp, Y)

            pred = hyp.data.max(dim=1)[1]

            num_corr += (pred == y).sum()
            num_tot += pred.size(0)

            val_loss += cost.item()

        val_accuracy = 0

        if num_tot > 0:
            val_accuracy =  num_corr / num_tot  

        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)

        if val_accuracy != 0:
            print("Epoch {}/{}, train_loss: {:.4f}, val_loss: {:.4f}, train_accuracy: {:.4f}, val_accuracy: {:.4f}"
                .format(epoch+1, n_epochs, train_loss, val_loss, train_accuracy, val_accuracy))
        else:
            print("Epoch {}/{}, train_loss: {:.4f}, train_accuracy: {:.4f}"
                .format(epoch+1, n_epochs, train_loss, train_accuracy))

    tl = np.array(train_losses).flatten()
    ta = np.array(train_accuracies).flatten()
    
    train_losses = tl
    train_accuracies = ta
    
    return model, train_losses, val_losses, train_accuracies, val_accuracies

In [131]:
num_epochs = 20
loss_fn = nn.CrossEntropyLoss()
lr = 0.001
opt_fn = torch.optim.Adam(network.parameters(),lr=lr)

In [132]:
history = train_model(num_epochs, network, train_loader, valid_loader, loss_fn, opt_fn, lr)
model, train_losses, val_losses, train_accuracies, val_accuracies = history

Epoch 1/20, train_loss: 222.1029, val_loss: 45.3543, train_accuracy: 0.8908, val_accuracy: 0.8977
Epoch 2/20, train_loss: 139.7828, val_loss: 32.0710, train_accuracy: 0.9599, val_accuracy: 0.9698
Epoch 3/20, train_loss: 126.1692, val_loss: 31.3163, train_accuracy: 0.9709, val_accuracy: 0.9697
Epoch 4/20, train_loss: 124.5815, val_loss: 31.3818, train_accuracy: 0.9713, val_accuracy: 0.9708
Epoch 5/20, train_loss: 122.5610, val_loss: 32.2256, train_accuracy: 0.9713, val_accuracy: 0.9673
Epoch 6/20, train_loss: 121.2661, val_loss: 31.3177, train_accuracy: 0.9719, val_accuracy: 0.9700
Epoch 7/20, train_loss: 119.7145, val_loss: 30.6741, train_accuracy: 0.9720, val_accuracy: 0.9711
Epoch 8/20, train_loss: 118.6946, val_loss: 30.8748, train_accuracy: 0.9721, val_accuracy: 0.9697
Epoch 9/20, train_loss: 118.0018, val_loss: 30.5011, train_accuracy: 0.9720, val_accuracy: 0.9702
Epoch 10/20, train_loss: 116.1812, val_loss: 30.1433, train_accuracy: 0.9724, val_accuracy: 0.9704
Epoch 11/20, train_

In [133]:
tens = []
pred = []
batch = []
actual = []

In [134]:
for i, (x, y) in enumerate(valid_loader):
    X = x.to(device=device)
    Y = y.to(device=device)
    Y = Y.squeeze()
    hyp = network(X)
    tens.append(hyp.data.max(dim=1)[1])
    batch.append(Y)

In [135]:
for t in tens:
    inter = t.numpy()
    pred = np.append(pred, inter)

pred

array([0., 0., 0., ..., 1., 0., 0.])

In [136]:
for t in batch:
    inter = t.numpy()
    actual = np.append(actual, inter)

actual = pd.Series(actual)
actual

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
9896    0.0
9897    0.0
9898    1.0
9899    0.0
9900    0.0
Length: 9901, dtype: float64

In [137]:
scores(pred,actual)

AUC-ROC: 0.8958510037929814
SMR: 1.1092896174863387


In [138]:
mimic_x_tensor = torch.Tensor(X_m.values.astype(np.float32))
mimic_y_tensor = torch.Tensor(Y_m.values.astype(np.float32))
mimic_y_tensor = mimic_y_tensor.long()
mimic_tensor = data_utils.TensorDataset(mimic_x_tensor, mimic_y_tensor)
mimic_loader = data_utils.DataLoader(dataset = mimic_tensor, batch_size = batch_size, shuffle = True)

eicu_x_tensor = torch.Tensor(X_e.values.astype(np.float32))
eicu_y_tensor = torch.Tensor(Y_e.values.astype(np.float32))
eicu_y_tensor = eicu_y_tensor.long()
eicu_tensor = data_utils.TensorDataset(eicu_x_tensor, eicu_y_tensor)
eicu_loader = data_utils.DataLoader(dataset = eicu_tensor, batch_size = batch_size, shuffle = True)

In [139]:
history = train_model(num_epochs, network, mimic_loader, eicu_loader, loss_fn, opt_fn, lr)
model, train_losses, val_losses, train_accuracies, val_accuracies = history

Epoch 1/20, train_loss: 140.1740, val_loss: 82.6697, train_accuracy: 0.9731, val_accuracy: 0.8359
Epoch 2/20, train_loss: 140.2334, val_loss: 85.4183, train_accuracy: 0.9732, val_accuracy: 0.8351
Epoch 3/20, train_loss: 138.4982, val_loss: 87.9455, train_accuracy: 0.9733, val_accuracy: 0.8396
Epoch 4/20, train_loss: 138.3496, val_loss: 88.9449, train_accuracy: 0.9732, val_accuracy: 0.8284
Epoch 5/20, train_loss: 137.8287, val_loss: 90.6401, train_accuracy: 0.9736, val_accuracy: 0.8251
Epoch 6/20, train_loss: 139.9392, val_loss: 86.9536, train_accuracy: 0.9735, val_accuracy: 0.8295
Epoch 7/20, train_loss: 139.1460, val_loss: 84.4417, train_accuracy: 0.9742, val_accuracy: 0.8393
Epoch 8/20, train_loss: 136.1653, val_loss: 87.8173, train_accuracy: 0.9737, val_accuracy: 0.8347
Epoch 9/20, train_loss: 136.1471, val_loss: 86.7845, train_accuracy: 0.9738, val_accuracy: 0.8353
Epoch 10/20, train_loss: 136.0586, val_loss: 86.2785, train_accuracy: 0.9737, val_accuracy: 0.8374
Epoch 11/20, train_

In [140]:
tens = []
pred = []
batch = []
actual = []

In [141]:
for i, (x, y) in enumerate(eicu_loader):
    X = x.to(device=device)
    Y = y.to(device=device)
    Y = Y.squeeze()
    hyp = network(X)
    tens.append(hyp.data.max(dim=1)[1])
    batch.append(Y)

In [142]:
for t in tens:
    inter = t.numpy()
    pred = np.append(pred, inter)

pred

array([0., 0., 0., ..., 0., 1., 0.])

In [143]:
for t in batch:
    inter = t.numpy()
    actual = np.append(actual, inter)

actual = pd.Series(actual)
actual

0       0.0
1       1.0
2       0.0
3       0.0
4       0.0
       ... 
6249    1.0
6250    0.0
6251    0.0
6252    1.0
6253    0.0
Length: 6254, dtype: float64

In [144]:
scores(pred,actual)

AUC-ROC: 0.7352201059623201
SMR: 0.8866141732283465
